<a href="https://colab.research.google.com/github/AlejandroVillazonG/Tareas_INF395/blob/main/T2/2_subclases_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<center><img src="https://www.inf.utfsm.cl/images/slides/Departamento-de-Informtica_HORIZONTAL.png" title="Title text" width= 800 /></center>
<hr style="height:2px;border:none"/>
<H1 align='center'> DESAFÍO TAREA 2</H1>

<H3 align='center'> INF395 2023-2 </H3>
<hr style="height:2px;border:none"/>

## Integrantes:
* Joaquín Aguirre (201910031-9)
* Alejandro Villazón (201910009-2)
* Dominique Yessouroun (201910005-K)

Importamos las librerías necesarias:

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

import os
import json
from sklearn.preprocessing import StandardScaler

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.applications import ResNet50

import warnings
# Ignorar las advertencias
warnings.filterwarnings("ignore")

Definimos funciones auxiliares:

In [2]:
def preprocess_image(main_path, path_image, image_size):
    image = keras.utils.load_img(main_path + path_image)
    image = image.resize((image_size, image_size))
    return keras.utils.img_to_array(image, dtype='uint8')

def preprocess_train_test_set(main_path, paths, image_size):
    return np.array([preprocess_image(main_path, path_image, image_size) for path_image in paths])

In [3]:
def create_macro_f1(y_true, y_pred, num_classes):
    """
    Calcula el Macro F1-Score.
    """
    y_pred = tf.one_hot(tf.argmax(y_pred, axis=1), depth=num_classes)

    tp = tf.cast(tf.math.count_nonzero(y_pred * y_true, axis=1), tf.float32)
    fp = tf.cast(tf.math.count_nonzero(y_pred * (1-y_true), axis=1), tf.float32)
    fn = tf.cast(tf.math.count_nonzero((1-y_pred) * y_true, axis=1), tf.float32)

    precision = tp / (tp + fp)
    recall = tp / (tp + fn)

    f1 = 2*precision*recall / (precision + recall)
    f1 = tf.where(tf.math.is_nan(f1), tf.zeros_like(f1), f1)

    return tf.reduce_mean(f1)

def transform_dishes(original_dishes, mapping_dishes, category_mapping):
    transformed_dishes = {}
    for original_key, dish in original_dishes.items():
        for category_key, category_value in category_mapping.items():
            if dish in mapping_dishes[category_value].values():
                # Encontrar la clave interna dentro del diccionario de categoría
                internal_key = [int(k) for k, v in mapping_dishes[category_value].items() if v == dish][0]
                # Convertir la clave original a int para guardarla en el resultado
                original_key_int = int(original_key)
                # Asegurar que la categoría exista en el diccionario resultante
                if category_key not in transformed_dishes:
                    transformed_dishes[category_key] = {}
                # Agregar el mapeo al diccionario resultante
                transformed_dishes[category_key][original_key_int] = internal_key
    return transformed_dishes

def create_model(image_size, num_classes):
  cnn_model = ResNet50(weights='imagenet', include_top=False, input_shape=(image_size, image_size, 3))
  for layer in cnn_model.layers:
      layer.trainable = False
  model = keras.Sequential([
      cnn_model,
      layers.GlobalAveragePooling2D(), #
      layers.Reshape((1, -1)), #
      layers.LSTM(256), #
      # layers.Conv2D(512, 2, activation='relu'),
      layers.Flatten(),
      layers.Dense(128, activation='relu'),
      layers.Dropout(0.5),
      layers.Dense(num_classes, activation='softmax')
  ])
  def macro_f1(y_true, y_pred):
    return create_macro_f1(y_true, y_pred, num_classes)

  model.compile(optimizer=keras.optimizers.AdamW(),
                loss=['categorical_crossentropy'],
                metrics=[macro_f1])
  return model

In [4]:
E_S = keras.callbacks.EarlyStopping(monitor='val_macro_f1', mode='max',
                                    patience=10, restore_best_weights=True)

Nos conectamos al drive:

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
path = '/content/drive/MyDrive/INF395/T2/'

## Preprocesar imágenes

In [7]:
!unzip '{path}inf-395-tarea-2.zip'  > /dev/null

In [8]:
main_path = '/content/'

df_train = pd.read_csv(main_path+'train.csv')

Definimos el tamaño que tendran las imágenes.

In [9]:
image_size = 225

In [10]:
%%time
X = preprocess_train_test_set(main_path, df_train['path'], image_size)

CPU times: user 6min 5s, sys: 16.4 s, total: 6min 21s
Wall time: 6min 20s


# NN para subclases

In [11]:
with open(path + "hierarchy_dict.json", "r") as archivo:
    hierarchy_dict = json.load(archivo)

with open(path + "food_categories_dict.json", "r") as archivo:
    food_categories_dict = json.load(archivo)

with open(path + "dish_dict.json", "r") as archivo:
    dish_dict = json.load(archivo)

In [12]:
transformed_dishes = transform_dishes(dish_dict, hierarchy_dict, food_categories_dict)

inverse_transformed_dishes = {}

for superclase, dictt in transformed_dishes.items():
  inverse_transformed_dishes[superclase] = {value: key for key, value in dictt.items()}

## Superclase 1

In [13]:
superclase = 1

In [14]:
n_subclasses = len(transformed_dishes[str(superclase)])
model_sub = create_model(image_size, n_subclasses)

model_sub.summary()

94765736/94765736 [==============================] - 3s 0us/step
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 8, 8, 2048)        23587712  
                                                                 
 global_average_pooling2d (  (None, 2048)              0         
 GlobalAveragePooling2D)                                         
                                                                 
 reshape (Reshape)           (None, 1, 2048)           0         
                                                                 
 lstm (LSTM)                 (None, 256)               2360320   
                                                                 
 flatten (Flatten)           (None, 256)               0         
                                                                 
 dense (Dense)               (None, 128)               32

In [15]:
y = df_train.loc[df_train['food_category'] == int(superclase), 'dish'].map(transformed_dishes[str(superclase)])

X = X[y.index]

In [16]:
X, X_val_sub, y, y_val_sub = train_test_split(X, y, test_size=0.1,
                                              random_state=42, stratify=y)

In [17]:
y = tf.one_hot(y, depth= n_subclasses)
y_val_sub = tf.one_hot(y_val_sub, depth= n_subclasses)

In [18]:
history = model_sub.fit(X, y,
                        batch_size = 128,
                        epochs = 200,
                        callbacks=[E_S],
                        verbose = 1,
                        validation_data = (X_val_sub, y_val_sub)
                        )

Epoch 1/200
111/111 [==============================] - 34s 164ms/step - loss: 2.2188 - macro_f1: 0.3433 - val_loss: 1.5950 - val_macro_f1: 0.5284
Epoch 2/200
111/111 [==============================] - 15s 136ms/step - loss: 1.6278 - macro_f1: 0.5104 - val_loss: 1.4557 - val_macro_f1: 0.5687
Epoch 3/200
111/111 [==============================] - 15s 135ms/step - loss: 1.4302 - macro_f1: 0.5662 - val_loss: 1.3234 - val_macro_f1: 0.6131
Epoch 4/200
111/111 [==============================] - 15s 135ms/step - loss: 1.3129 - macro_f1: 0.6034 - val_loss: 1.2858 - val_macro_f1: 0.6213
Epoch 5/200
111/111 [==============================] - 15s 134ms/step - loss: 1.2207 - macro_f1: 0.6286 - val_loss: 1.2789 - val_macro_f1: 0.6054
Epoch 6/200
111/111 [==============================] - 15s 135ms/step - loss: 1.1021 - macro_f1: 0.6614 - val_loss: 1.2571 - val_macro_f1: 0.6228
Epoch 7/200
111/111 [==============================] - 15s 136ms/step - loss: 1.0283 - macro_f1: 0.6855 - val_loss: 1.2347 -

In [19]:
model_sub.save(path+f'/NNs/NN_subclases_{superclase}.keras')